<a href="https://colab.research.google.com/github/kev-in-xu/Phylo-Exceptions/blob/main/Organisms_lacking_Dmc1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv

from google.colab import files

!pip install biopython
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
from Bio import Entrez

import requests

#%env STORAGE_FILE_PATH=/content/drive/MyDrive/Greene_lab/
%env STORAGE_FILE_PATH=/content/drive/MyDrive/Greene-Lab/lacking_organisms/nov_data/
from google.colab import drive
drive.mount('/content/drive')
# ! ls $STORAGE_FILE_PATH


Entrez.api_key = "a5d84a29ce91f0aa01001ad64a26fc145208"
Entrez.email = "kkx2102@columbia.edu"
path = %env STORAGE_FILE_PATH

### Workbook upload / parsing


In [ ]:
path = %env STORAGE_FILE_PATH
rad51 = pd.read_excel(path + "Rad51_Workbook.xlsx")
dmc1 = pd.read_excel(path + "Dmc1_Workbook.xlsx")

In [ ]:
f = open(path + "Dmc1_seqdump.txt", "r")
sequences = []
current_sequence = {"header": "", "sequence": ""}

# Open and read the FASTA file
with open(path + "Dmc1_seqdump.txt", "r") as fasta_file:
    for line in fasta_file:
        line = line.strip()
        if line.startswith(">"):  # Header line
            if current_sequence["header"]:
                sequences.append(current_sequence.copy())
            current_sequence["header"] = line[1:]  # Remove '>' from header
            current_sequence["sequence"] = ""
        else:  # Sequence line
            current_sequence["sequence"] += line

# Don't forget to add the last sequence after the loop
if current_sequence["header"]:
    sequences.append(current_sequence)

In [ ]:
Dmc1_orgs = []

for seq in sequences:
  for a in seq["header"].split():
    if a.startswith("["):
      Dmc1_orgs.append(a[1:])

dmc1_genus = list(set([org for org in Dmc1_orgs]))

### pBLAST Automation

In [ ]:
### edited from ChatGPT

yeast_seq = "MSVTGTEIDSDTAKNILSVDELQNYGINASDLQKLKSGGIYTVNTVLSTTRRHLCKIKGLSEVKVEKIKEAAGKIIQVGFIPATVQLDIRQRVYSLSTGSKQLDSILGGGIMTMSITEVFGEFRCGKTQMSHTLCVTTQLPREMGGGEGKVAYIDTEGTFRPERIKQIAEGYELDPESCLANVSYARALNSEHQMELVEQLGEELSSGDYRLIVVDSIMANFRVDYCGRGELSERQQKLNQHLFKLNRLAEEFNVAVFLTNQVQSDPGASALFASADGRKPIGGHVLAHASATRILLRKGRGDERVAKLQDSPDMPEKECVYVIGEKGITDSSD"

human_seq = "MKEDQVVAEEPGFQDEEESLFQDIDLLQKHGINVADIKKLKSVGICTIKGIQMTTRRALCNVKGLSEAKVDKIKEAANKLIEPGFLTAFEYSEKRKMVFHITTGSQEFDKLLGGGIESMAITEAFGEFRTGKTQLSHTLCVTAQLPGAGGYPGGKIIFIDTENTFRPDRLRDIADRFNVDHDAVLDNVLYARAYTSEHQMELLDYVAAKFHEEAGIFKLLIIDSIMALFRVDFSGRGELAERQQKLAQMLSRLQKISEEYNVAVFVTNQMTADPGATMTFQADPKKPIGGHILAHASTTRISLRKGRGELRIAKIYDSPEMPENEATFAITAGGIGDAKE"

# Replace this with your actual query sequence in FASTA format
# query_seq = ">Query\n" + yeast_seq
query_seq = ">Query\n" + human_seq

# Perform pBLAST search
result_handle = NCBIWWW.qblast("blastp", "refseq_protein", query_seq, hitlist_size=500)

# Parse the BLAST results
blast_records = NCBIXML.parse(result_handle)

# List to store organisms without DMC1 or HSLIM15 entry
organisms_without_entry = []
organisms = []

# Iterate through BLAST records
for record in blast_records:
    for alignment in record.alignments:
        organisms.append(alignment.title.split("[")[1].split("]")[0])

#alignment_title = alignment.title
# Check if alignment title contains DMC1 or HSLIM15
#if "DMC1" not in alignment_title.upper() and "HSLIM15" not in alignment_title.upper():
 #   organism = alignment_title.split("[")[1].split("]")[0]
  #  organisms_without_entry.append(organism)


# Print organisms without DMC1/HSLIM15 entry
print("Organisms without DMC1 or HSLIM15 entry:")
for organism in organisms_without_entry:
    print(organism)


In [ ]:
len(record.alignments)

In [ ]:
alignment.hsps[0].expect

In [ ]:
organisms

### BLASTing exceptions

In [ ]:
#path = %env STORAGE_FILE_PATH
# opens excel files
#rad51 = pd.read_excel(path + "Rad51_Workbook.xlsx")
# opens files
#f = open(path + "Dmc1_seqdump.txt", "r")

In [ ]:
path = %env STORAGE_FILE_PATH
Dmc1branch = open(path + "Dmc1branch_240-520_85.fa.clstr", "r")

pblastSeqs = []
current_sequence = {"header": "","accession no.": "",
                    "organism": "", "sequence": "", "length": ""}

# Open and read the pBLAST FASTA file
with open(path + "Rad51_1e-25_7693.fasta", "r") as pblastResult:
    for line in pblastResult:
        line = line.strip()
        if line.startswith(">"):  # Header line
            if current_sequence["header"]:
              current_sequence["length"] = len(current_sequence["sequence"])
              pblastSeqs.append(current_sequence.copy())
            current_sequence["header"] = line[1:]  # Remove '>' from header
            current_sequence["sequence"] = ""
            current_sequence["organism"] = line[line.find("[")+1:line.rfind("]")]
            current_sequence["accession no."] = line[1:line.find(" ")]
        else:  # Sequence line
            current_sequence["sequence"] += line

# Don't forget to add the last sequence after the loop
if current_sequence["header"]:
    pblastSeqs.append(current_sequence)

master = pd.DataFrame.from_dict(pblastSeqs)


###DMC1###
Dmc1Clusters = []
current_cluster = {"accession no.": "", "protein": ""}

# Open and read the Dmc1 FASTA file
with open(path + "Dmc1branch_240-520_85.fa.clstr", "r") as Dmc1File:
    for line in Dmc1File:
        line = line.strip()
        if not line.startswith(">"):  # non-cluster line
            current_cluster["accession no."] = line[line.find(">")+1:line.find(".")]
            current_cluster["protein"] = "Dmc1"
            Dmc1Clusters.append(current_cluster.copy())
Dmc1 = pd.DataFrame.from_dict(Dmc1Clusters)


###RAD51###
Rad51Clusters = []
current_cluster = {"accession no.": "", "protein": ""}

# Open and read the Dmc1 FASTA file
with open(path + "Rad51branch_240-520_85.fa.clstr", "r") as Rad51File:
    for line in Rad51File:
        line = line.strip()
        if not line.startswith(">"):  # non-cluster line
            current_cluster["accession no."] = line[line.find(">")+1:line.find(".")]
            current_cluster["protein"] = "Rad51"
            Rad51Clusters.append(current_cluster.copy())
Rad51 = pd.DataFrame.from_dict(Rad51Clusters)

In [ ]:
# Merges Columns to include protein data
master=master.merge(Rad51, how="left", left_on="accession no.", right_on="accession no.")
master=master.merge(Dmc1, how="left", left_on="accession no.", right_on="accession no.")

def combine_columns(row):
    return row["protein_x"] if pd.isna(row["protein_y"]) else row["protein_y"]

master['protein'] = master.apply(combine_columns, axis=1)
master.drop(columns=["protein_x","protein_y"])

In [ ]:
Rad51_orgs = list(master["organism"][master["protein"] == "Rad51"])
Dmc1_orgs = list(master["organism"][master["protein"] == "Dmc1"])

In [ ]:
Rad51_orgs = list(set([org.split(" ")[0] for org in Rad51_orgs]))
Dmc1_orgs = list(set([org.split(" ")[0] for org in Dmc1_orgs]))

Rad51_only = [org for org in Rad51_orgs if org not in Dmc1_orgs]
Dmc1_only = [org for org in Dmc1_orgs if org not in Rad51_orgs]

In [ ]:
Rad51_only

#### Mike's Dataset

In [ ]:
###RAD51###
Rad51Clusters = []
current_cluster = {"accession no.": "", "protein": ""}

# Open and read the Dmc1 FASTA file
with open(path + "Rad51branch_240-520_85.fa.clstr", "r") as Rad51File:
    for line in Rad51File:
        line = line.strip()
        if not line.startswith(">"):  # non-cluster line
            current_cluster["accession no."] = line[line.find(">")+1:line.find(".")]
            current_cluster["protein"] = "Rad51"
            Rad51Clusters.append(current_cluster.copy())
Rad51 = pd.DataFrame.from_dict(Rad51Clusters)

### Just using NCBI Taxonomy

In [ ]:
### PARSING ALL PHYLOGENETIC ORDERS FROM GIVEN FILES ###
path = %env STORAGE_FILE_PATH
orders = []

with open(path + "Metazoa_orders.txt", "r") as taxonomy:
  for line in taxonomy:
    line = line.strip()
    if line.find("-") != -1 and line.find("---") == -1:
      orders.append(line[line.find("-")+1:])

In [ ]:
##### GETS TAXIDS FROM LIST OF TAXONOMY RANKS ######

IDs = []
ID = {"organism": "", "txid": "", "count": ""}

# Iterates through organisms
for organism_name in orders:
  # Construct the URL for the NCBI Taxonomy API
  base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
  search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name

  # Send a GET request to the API
  response = requests.get(search_url)
  data = response.text

  # Extract the TaxID from the response
  if data.find("<Id>")!=-1:
    taxid = data[data.find("<Id>")+4:data.find("</Id>")]
    print(f"TaxID for {organism_name}: {taxid}")
  else:
    taxid = ""
    #print(f"TaxID not found for {organism_name}")

  ID["organism"]=organism_name
  ID["txid"]=taxid + "[ORGN]"

  # Construct the URL for the NCBI Taxonomy API
  base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
  search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name + "[Organism]"

  # Send a GET request to the API
  response = requests.get(search_url)
  data = response.text

  # Extract the TaxID from the response
  if data.find("<Count>")!=-1:
    count = data[data.find("<Count>")+7:data.find("</Count>")]
    print(f"Count for {organism_name}: {count}")
  else:
    count = 0
    #print(f"TaxID not found for {organism_name}")

  ID["count"]=count
  IDs.append(ID.copy())

In [ ]:
IDs

In [ ]:
base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name

# Send a GET request to the API
response = requests.get(search_url)
data = response.text

# Extract the TaxID from the response
if data.find("<Id>")!=-1:
  taxid = data[data.find("<Id>")+4:data.find("</Id>")]
  print(f"TaxID for {organism_name}: {taxid}")

In [ ]:
### Exports Taxids to csv ####
import csv

field_names = ['organism', 'txid', 'count']

with open(path +'Names.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(IDs)

#### Read CSV

In [ ]:
### Reads existing csv for dictionary ###

path = %env STORAGE_FILE_PATH

IDs = []
with open(path +'Names.csv', 'r') as csvfile:
  csv_reader = csv.DictReader(csvfile)
  for row in csv_reader:
    IDs.append(row)

In [ ]:
zeroes = []

for org in IDs:
  if int(org["count"]) == 0 or org["txid"] == "[ORGN]":
    zeroes.append(org)


zero_orgs = []

for org in zeroes:
  zero_orgs.append(org["organism"])

In [ ]:
len(zero_orgs)

In [ ]:
##### GETS TAXIDS FROM LIST OF TAXONOMY RANKS ######

import time

IDs = []
ID = {"organism": "", "txid": "", "count": ""}

# Iterates through organisms
for organism_name in zero_orgs:
  # Construct the URL for the NCBI Taxonomy API
  base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
  search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name

  # Send a GET request to the API
  response = requests.get(search_url)
  data = response.text

  # Extract the TaxID from the response
  if data.find("<Id>")!=-1:
    taxid = data[data.find("<Id>")+4:data.find("</Id>")]
    print(f"TaxID for {organism_name}: {taxid}")
  else:
    taxid = ""
    print(f"TaxID not found for {organism_name}")

  ID["organism"]=organism_name
  ID["txid"]=taxid + "[ORGN]"

  # Construct the URL for the NCBI Taxonomy API
  base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
  search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name + "[Organism]"

  # Send a GET request to the API
  response = requests.get(search_url)
  data = response.text

  # Extract the TaxID from the response
  if data.find("<Count>")!=-1:
    count = data[data.find("<Count>")+7:data.find("</Count>")]
    print(f"Count for {organism_name}: {count}")
  else:
    count = 0
    print(f"Count not found for {organism_name}")

  ID["count"]=count
  IDs.append(ID.copy())
  #time.sleep(1)

In [ ]:
IDs

In [ ]:
IDs[590:]

In [ ]:
yeast_seq = "MSVTGTEIDSDTAKNILSVDELQNYGINASDLQKLKSGGIYTVNTVLSTTRRHLCKIKGLSEVKVEKIKEAAGKIIQVGFIPATVQLDIRQRVYSLSTGSKQLDSILGGGIMTMSITEVFGEFRCGKTQMSHTLCVTTQLPREMGGGEGKVAYIDTEGTFRPERIKQIAEGYELDPESCLANVSYARALNSEHQMELVEQLGEELSSGDYRLIVVDSIMANFRVDYCGRGELSERQQKLNQHLFKLNRLAEEFNVAVFLTNQVQSDPGASALFASADGRKPIGGHVLAHASATRILLRKGRGDERVAKLQDSPDMPEKECVYVIGEKGITDSSD"

human_seq = "MKEDQVVAEEPGFQDEEESLFQDIDLLQKHGINVADIKKLKSVGICTIKGIQMTTRRALCNVKGLSEAKVDKIKEAANKLIEPGFLTAFEYSEKRKMVFHITTGSQEFDKLLGGGIESMAITEAFGEFRTGKTQLSHTLCVTAQLPGAGGYPGGKIIFIDTENTFRPDRLRDIADRFNVDHDAVLDNVLYARAYTSEHQMELLDYVAAKFHEEAGIFKLLIIDSIMALFRVDFSGRGELAERQQKLAQMLSRLQKISEEYNVAVFVTNQMTADPGATMTFQADPKKPIGGHILAHASTTRISLRKGRGELRIAKIYDSPEMPENEATFAITAGGIGDAKE"


# List to store organisms without DMC1 or HSLIM15 entry
containsDmc1 = []
lacksDmc1 = []
fewerThan100 = []

for ID in IDs[599:600]:
  hasDmc1 = False

  if int(ID["count"]) > 100:
    # Replace this with your actual query sequence in FASTA format
    # query_seq = ">Query\n" + yeast_seq
    query_seq = ">Query\n" + human_seq
    order = ID["organism"] + "[ORGN]"

    # Perform pBLAST search
    result = NCBIWWW.qblast("blastp", "nr", query_seq, entrez_query=order, hitlist_size=10)

    # Parse the BLAST results
    rad51_parsed = NCBIXML.parse(result)

    # Iterate through BLAST records
    for record in rad51_parsed:
      for alignment in record.alignments:
        if (alignment.title.find("dmc1") != -1 or alignment.title.find("Dmc1") != -1
            or alignment.title.find("DMC1") != -1or alignment.title.find("lim15") != -1
            or alignment.title.find("Lim15") != -1 or alignment.title.find("LIM15") != -1
            or alignment.hsps[0].expect < 1e-120): # THIS IS THE FILTER CRITERIA
          hasDmc1 = True
      if hasDmc1:
        containsDmc1.append(ID["organism"])
        print(ID["organism"] + " has Dmc1")
      else:
        lacksDmc1.append(ID["organism"])
        print(ID["organism"] + " lack Dmc1!")
  else:
    fewerThan100.append(ID["organism"])
    print(ID["organism"] + " has fewer than 100 entries")

with open(path + "output.txt", "w") as output:
  output.write('-----containsDmc1-----\n')
  for order in containsDmc1:
    output.write(order + "\n")
  output.write('\n-----lacksDmc1-----\n')
  for order in lacksDmc1:
    output.write(order + "\n")
  output.write('\n-----fewerThan100-----\n')
  for order in fewerThan100:
    output.write(str(order) + "\n")

files.download("output.txt")


## October Plan

In [ ]:
### Reads existing csv for dictionary ###

path = %env STORAGE_FILE_PATH

orders = []
with open(path +'plant_orders.csv', 'r') as csvfile:
  csv_reader = csv.DictReader(csvfile)
  for row in csv_reader:
    orders.append(row["Order"])

In [ ]:
orders

In [ ]:
import time

IDs = []
ID = {"organism": "", "txid": ""}

# Iterates through organisms
for organism_name in orders:
  time.sleep(0.25)

  # Construct the URL for the NCBI Taxonomy API
  base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
  search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name

  # Send a GET request to the API
  response = requests.get(search_url)
  data = response.text

  # Extract the TaxID from the response
  if data.find("<Id>")!=-1:
    taxid = data[data.find("<Id>")+4:data.find("</Id>")]
    print(f"TaxID for {organism_name}: {taxid}")
  else:
    # Construct the URL for the NCBI Taxonomy API
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    search_url = base_url + "esearch.fcgi?db=taxonomy&term=" + organism_name

    # Send a GET request to the API
    response = requests.get(search_url)
    data = response.text

    # Extract the TaxID from the response
    if data.find("<Id>")!=-1:
      taxid = data[data.find("<Id>")+4:data.find("</Id>")]
      print(f"TaxID for {organism_name}: {taxid}")
    else:
      taxid = ""
      print(f"TaxID not found for {organism_name}")

  ID["organism"]=organism_name
  ID["txid"]=taxid + "[ORGN]"

  IDs.append(ID.copy())

In [ ]:
IDs

In [ ]:
import csv

field_names = ['organism', 'txid']

with open(path +'plant_orders_txid.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = field_names)
    writer.writeheader()
    writer.writerows(IDs)

In [ ]:
files.download(path +'plant_orders_txid.csv')

### Txid CSV

In [ ]:
### Reads existing csv for dictionary ###

path = %env STORAGE_FILE_PATH

IDs = []
order = {"organism":"","txid":""}
with open(path +'plant_orders_txid.csv', 'r') as csvfile:
  csv_reader = csv.DictReader(csvfile)
  for row in csv_reader:
    order["organism"] = row["organism"]
    order["txid"] = row["txid"]
    IDs.append(order.copy())

In [ ]:
len(IDs)

In [ ]:
# List to store organisms without DMC1 or HSLIM15 entry
containsDmc1 = []
lacksDmc1 = []
fewerThan100 = []

yeast_seq = "MSVTGTEIDSDTAKNILSVDELQNYGINASDLQKLKSGGIYTVNTVLSTTRRHLCKIKGLSEVKVEKIKEAAGKIIQVGFIPATVQLDIRQRVYSLSTGSKQLDSILGGGIMTMSITEVFGEFRCGKTQMSHTLCVTTQLPREMGGGEGKVAYIDTEGTFRPERIKQIAEGYELDPESCLANVSYARALNSEHQMELVEQLGEELSSGDYRLIVVDSIMANFRVDYCGRGELSERQQKLNQHLFKLNRLAEEFNVAVFLTNQVQSDPGASALFASADGRKPIGGHVLAHASATRILLRKGRGDERVAKLQDSPDMPEKECVYVIGEKGITDSSD"

human_seq = "MKEDQVVAEEPGFQDEEESLFQDIDLLQKHGINVADIKKLKSVGICTIKGIQMTTRRALCNVKGLSEAKVDKIKEAANKLIEPGFLTAFEYSEKRKMVFHITTGSQEFDKLLGGGIESMAITEAFGEFRTGKTQLSHTLCVTAQLPGAGGYPGGKIIFIDTENTFRPDRLRDIADRFNVDHDAVLDNVLYARAYTSEHQMELLDYVAAKFHEEAGIFKLLIIDSIMALFRVDFSGRGELAERQQKLAQMLSRLQKISEEYNVAVFVTNQMTADPGATMTFQADPKKPIGGHILAHASTTRISLRKGRGELRIAKIYDSPEMPENEATFAITAGGIGDAKE"


for ID in IDs[74:]:
  hasDmc1 = False

  # Replace this with your actual query sequence in FASTA format
  query_seq = ">Query\n" + yeast_seq
  #query_seq = ">Query\n" + human_seq
  order = ID["organism"] + "[ORGN]"

  # Perform pBLAST search
  result = NCBIWWW.qblast("blastp", "nr", query_seq, entrez_query=order, hitlist_size=10)

  # Parse the BLAST results
  rad51_parsed = NCBIXML.parse(result)

  # Iterate through BLAST records
  for record in rad51_parsed:
    for alignment in record.alignments:
      if (alignment.title.find("dmc1") != -1 or alignment.title.find("Dmc1") != -1
          or alignment.title.find("DMC1") != -1 or alignment.title.find("lim15") != -1
          or alignment.title.find("Lim15") != -1 or alignment.title.find("LIM15") != -1
          or alignment.hsps[0].expect < 1e-120): # THIS IS THE FILTER CRITERIA
        hasDmc1 = True
    if hasDmc1:
      containsDmc1.append(ID["organism"])
      print(ID["organism"] + " has Dmc1")
    else:
      lacksDmc1.append(ID["organism"])
      print(ID["organism"] + " lack Dmc1")

with open(path + "output.txt", "w") as output:
  output.write('-----containsDmc1-----\n')
  for order in containsDmc1:
    output.write(order + "\n")
  output.write('\n-----lacksDmc1-----\n')
  for order in lacksDmc1:
    output.write(order + "\n")
  output.write('\n-----fewerThan100-----\n')
  for order in fewerThan100:
    output.write(str(order) + "\n")

files.download(path + "output.txt")

In [ ]:
### RAD51 CHECKLIST
# List to store organisms without Rad51 entry
containsRad51 = []
lacksRad51 = []
fewerThan100 = []

yeast_Rad51 = "MSQVQEQHISESQLQYGNGSLMSTVPADLSQSVVDGNGNGSSEDIEATNGSGDGGGLQEQAEAQGEMEDEAYDEAALGSFVPIEKLQVNGITMADVKKLRESGLHTAEAVAYAPRKDLLEIKGISEAKADKLLNEAARLVPMGFVTAADFHMRRSELICLTTGSKNLDTLLGGGVETGSITELFGEFRTGKSQLCHTLAVTCQIPLDIGGGEGKCLYIDTEGTFRPVRLVSIAQRFGLDPDDALNNVAYARAYNADHQLRLLDAAAQMMSESRFSLIVVDSVMALYRTDFSGRGELSARQMHLAKFMRALQRLADQFGVAVVVTNQVVAQVDGGMAFNPDPKKPIGGNIMAHSSTTRLGFKKGKGCQRLCKVVDSPCLPEAECVFAIYEDGVGDPREEDE"
human_Rad51 = "MAMQMQLEANADTSVEEESFGPQPISRLEQCGINANDVKKLEEAGFHTVEAVAYAPKKELINIKGISEAKADKILAEAAKLVPMGFTTATEFHQRRSEIIQITTGSKELDKLLQGGIETGSITEMFGEFRTGKTQICHTLAVTCQLPIDRGGGEGKAMYIDTEGTFRPERLLAVAERYGLSGSDVLDNVAYARAFNTDHQTQLLYQASAMMVESRYALLIVDSATALYRTDYSGRGELSARQMHLARFLRMLLRLADEFGVAVVITNQVVAQVDGAAMFAADPKKPIGGNIIAHASTTRLYLRKGRGETRICKIYDSPCLPEAEAMFAINADGVGDAKD"


for ID in IDs[72:]:
  hasRad51 = False

  # Replace this with your actual query sequence in FASTA format
  # query_seq = ">Query\n" + yeast_Rad51
  query_seq = ">Query\n" + human_Rad51
  order = ID["organism"] + "[ORGN]"

  # Perform pBLAST search
  result = NCBIWWW.qblast("blastp", "nr", query_seq, entrez_query=order, hitlist_size=10)

  # Parse the BLAST results
  rad51_parsed = NCBIXML.parse(result)

  # Iterate through BLAST records
  for record in rad51_parsed:
    for alignment in record.alignments:
      if (alignment.title.find("Rad51") != -1 or alignment.title.find("RAD51") != -1
          or alignment.hsps[0].expect < 1e-120): # THIS IS THE FILTER CRITERIA
        hasRad51 = True
    if hasRad51:
      containsRad51.append(ID["organism"])
      print(ID["organism"] + " has Rad51")
    else:
      lacksRad51.append(ID["organism"])
      print(ID["organism"] + " lack Rad51!")

with open(path + "output.txt", "w") as output:
  output.write('-----containsRad51-----\n')
  for order in containsRad51:
    output.write(order + "\n")
  output.write('\n-----lacksRad51-----\n')
  for order in lacksRad51:
    output.write(order + "\n")
#  output.write('\n-----fewerThan100-----\n')
#  for order in fewerThan100:
#    output.write(str(order) + "\n")

files.download(path + "output.txt")

# November Plan

The plan is to download the top 3 sequences and headers of each species, do a manual sifting of results so that it becomes one sequence per order. Then do an alignment and FastTree pipeline to create the newick tree. Then put it into a pdf form and do the highlighting for visualization


In [ ]:
### Reads existing csv for dictionary ###

orders = []
with open(path +'metazoa_orders_pruned.csv', 'r') as csvfile:
  csv_reader = csv.DictReader(csvfile)
  for row in csv_reader:
    orders.append(row["Order"])

In [ ]:
### Reads master csv in case progress is dropped ###

master = pd.read_csv(path + "metazoa_data_nov.csv")
master

In [ ]:
data = {
          "Order": ["temp"],
          "Title": ["temp"],
          "Alignment Length": [0],
          "Alignment Sequence": ["temp"]
      }
master = pd.DataFrame(data)

In [ ]:
master

In [ ]:
orders[60:65]

In [ ]:
def fetch_protein_sequence(accession_number):
    try:
        # Fetch the protein record using the accession number
        handle = Entrez.efetch(db="protein", id=accession_number, rettype="fasta", retmode="text")
        record = handle.read()
        handle.close()
        return record
    except Exception as e:
        return str(e)

yeast_Rad51 = "MSQVQEQHISESQLQYGNGSLMSTVPADLSQSVVDGNGNGSSEDIEATNGSGDGGGLQEQAEAQGEMEDEAYDEAALGSFVPIEKLQVNGITMADVKKLRESGLHTAEAVAYAPRKDLLEIKGISEAKADKLLNEAARLVPMGFVTAADFHMRRSELICLTTGSKNLDTLLGGGVETGSITELFGEFRTGKSQLCHTLAVTCQIPLDIGGGEGKCLYIDTEGTFRPVRLVSIAQRFGLDPDDALNNVAYARAYNADHQLRLLDAAAQMMSESRFSLIVVDSVMALYRTDFSGRGELSARQMHLAKFMRALQRLADQFGVAVVVTNQVVAQVDGGMAFNPDPKKPIGGNIMAHSSTTRLGFKKGKGCQRLCKVVDSPCLPEAECVFAIYEDGVGDPREEDE"
human_seq = "MAMQMQLEANADTSVEEESFGPQPISRLEQCGINANDVKKLEEAGFHTVEAVAYAPKKELINIKGISEAKADKILAEAAKLVPMGFTTATEFHQRRSEIIQITTGSKELDKLLQGGIETGSITEMFGEFRTGKTQICHTLAVTCQLPIDRGGGEGKAMYIDTEGTFRPERLLAVAERYGLSGSDVLDNVAYARAFNTDHQTQLLYQASAMMVESRYALLIVDSATALYRTDYSGRGELSARQMHLARFLRMLLRLADEFGVAVVITNQVVAQVDGAAMFAADPKKPIGGNIIAHASTTRLYLRKGRGETRICKIYDSPCLPEAEAMFAINADGVGDAKD"


#query_seq = ">Query\n" + yeast_Rad51
query_seq = ">Query\n" + human_seq

for order in orders:
  #### SET UP LIST OF ORDERS FROM SPREADSHEET ANALYSIS
  order_query = order + "[ORGN]"

  # Perform pBLAST search
  result = NCBIWWW.qblast("blastp", "nr", query_seq, entrez_query=order_query, hitlist_size=10)

  # Parse the BLAST results
  rad51_parsed = NCBIXML.parse(result)

  # Extract information from XML parse
  for record in rad51_parsed:
    order_name = []
    title = []
    length = []
    subject = []

    for alignment in record.alignments:
      t = alignment.title

      order_name.append(order)
      title.append(t)
      length.append(alignment.hsps[0].align_length)

      # Parses accession number and uses Entrez to fetch protein sequence
      index = t.find("|")
      accession_number = t[index+1 : t[index+1:].find('|') + index+1]
      protein_sequence = fetch_protein_sequence(accession_number)
      subject.append(protein_sequence)

    data = {
          "Order": order_name,
          "Title": title,
          "Alignment Length": length,
          "Alignment Sequence": subject
      }
    df = pd.DataFrame(data)
    master = pd.concat([df,master], ignore_index=True)
    master.to_csv(path + "metazoa_data_nov.csv", index=False)

    print(order + " done...")

In [ ]:
order

In [ ]:
master.to_csv(path + "/nov_data/fungi_data_nov.csv", index=False)